In [265]:
import importlib
import functions as fn
importlib.reload(fn)
import pandas as pd

In [140]:
conn = psycopg2.connect(
           dbname="qdap_test",
           user="amt",
           password="your_password",
           host="192.168.2.23",
           port="5432"
       )

# Create a cursor object using the cursor() method
cursor = conn.cursor()


In [141]:
symbol = 'BANKNIFTY'  # Replace with your symbol
expiry_future = fn.fetch_expiries(cursor, symbol)

expiry_future['expiry'] = pd.to_datetime(expiry_future['expiry']).dt.date


In [142]:
for i, row in expiry_future.iterrows():
    print(i, row['expiry'], sep=' = ', end=' | ')
    if (i + 1) % 6 == 0:
        print()

0 = 2022-05-26 | 1 = 2022-06-30 | 2 = 2022-07-28 | 3 = 2022-08-25 | 4 = 2022-09-29 | 5 = 2022-10-27 | 
6 = 2022-11-24 | 7 = 2022-12-29 | 8 = 2023-01-25 | 9 = 2023-02-23 | 10 = 2023-03-29 | 11 = 2023-04-27 | 
12 = 2023-05-25 | 13 = 2023-06-28 | 14 = 2023-06-29 | 15 = 2023-07-27 | 16 = 2023-08-31 | 17 = 2023-09-28 | 
18 = 2023-10-26 | 19 = 2023-11-30 | 20 = 2023-12-28 | 21 = 2024-01-25 | 22 = 2024-02-29 | 23 = 2024-03-27 | 
24 = 2024-04-24 | 25 = 2024-05-29 | 26 = 2024-06-26 | 27 = 2024-07-31 | 

In [148]:
index = 20

# Fetch the futures data for the selected expiry date
futures = fn.fetch_futures(cursor, symbol,expiry_future.loc[index,'expiry'])
# Convert the 'date_timestamp' column to datetime if it exists
if 'id' in futures.columns:
    futures.drop(columns=['id'] , inplace=True) 
futures.drop_duplicates(inplace=True)


In [149]:
fn.fill_missings(futures)

In [170]:
fn.genrate_signals(futures)

In [180]:
# fig = plot_signals(futures)

# fig.show()

In [154]:
options = fn.fetch_options(cursor, symbol=symbol, expiry=expiry_future.loc[index,'expiry'])

In [213]:
calls , puts = fn.fetch_call_put(options)

In [267]:
trades = fn.generate_trades(futures,calls,puts )

# trades = fn.generate_trades_(futures,calls,puts,initial_capital=1000000)

In [268]:
print(len(trades))

282


In [258]:
columns_to_print = {
    'pos': 1, 'signal_time': 1, 'long_ema': 1, 'short_ema': 1, 'entry_time': 1,
    'strike': 1, 'entry_price': 1, 'exit_time': 1, 'exit_price': 1, 'pnl': 1,
    'type': 1, 'pnl_sum': 1, 'price': 1, 'logs': 1, 'movment': 1, 'sharpie_ratio': 1, 'drawdown': 1
}
fn.print_trades(trades, columns_to_print)

TypeError: list indices must be integers or slices, not str

In [171]:
# fig = fn.grpah(x=[i for i in range(len(trades))] , y=[i['pnl'] for i in trades])


In [172]:
# fig = fn.grpah(x=[i for i in range(len(trades))] , y=[i['pnl_sum'] for i in trades])

In [ ]:
drawdownval = fn.drawdown(trades)
print(drawdownval)

173600.0


In [ ]:
print(f"sharpie ratio = {fn.sharpie(trades)}")

sharpie ratio = 0.11523192994542368
